<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<a href="./00_index.ipynb"><< Index - Dashboarding with Jupyter Notebooks and Voila </a><span style="float:right;"><a href="./02_voila_dashboards.ipynb"> 02 - Introduction to Voila dashboards >></a></span>

# 3 - Voila dashboards - Example 1

* [What are Jupyter widgets?](#about)
* [Installing Jupyter widgets](#installation)
* [Jupyter widgets - an overview](#overview)
* [Jupyter widgets in action](#action)

<hr>

## Example - Interactive application of climate graphs

Required libraries:
* [Ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)
* [Plotly](https://plot.ly/) for interactive visualization [Registration required]
* [Widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)

In [42]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipyleaflet

from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import ee
import pandas as pd

##import plotly.plotly as py
#import plotly.graph_objs as go
#import chart_studio.plotly as py
import altair as alt

In [19]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [22]:
ee.Initialize()

### Load ERA5 monthly ImageCollection 

In [23]:
era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY')

### Process mean precipitation for each month based on entire time series

In [24]:
era5_monthly_img = era5_monthly.limit(1).first()
collection_img_proj = era5_monthly_img.select(0).projection()
era5_monthly_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'minimum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'maximum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'dewpoint_2m_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'total_precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_trans

In [25]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = era5_monthly.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    
    # if reducer function is applied to an image collection, the output does not have any projection information, as collection can contain \ 
    # images with different projection information. Thus, one can set the projection to each image
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
img_list[0].getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'minimum_2m_air_temperature_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'maximum_2m_air_temperature_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'dewpoint_2m_temperature_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'total_precipitation_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'surface_pressure_mean',
   'data_type': {'type': 'PixelType', 'precision': '

### Build an Image Collection of the resulting image list

In [26]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
meanMonths_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'mean_2m_air_temperature_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
    {'id': 'minimum_2m_air_temperature_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
    {'id': 'maximum_2m_air_temperature_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
    {'id': 'dewpoint_2m_temperature_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
    {'id': 'total_precipitation_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0, -180, 0, -0.

In [27]:
t2m_tmp = meanMonths_collection.select('mean_2m_air_temperature_mean').first()
tp_tmp = meanMonths_collection.select('total_precipitation_mean').first()

In [28]:
img_t2m = GetTileLayerUrl(t2m_tmp.visualize(min=250, max=330, palette=['#000080','#0000D9','#4000FF','#8000FF','#0080FF','#00FFFF','#00FF80','#80FF00','#DAFF00','#FFFF00','#FFF500','#FFDA00','#FFB000','#FFA400','#FF4F00','#FF2500','#FF0A00','#FF00FF']))
img_tp = GetTileLayerUrl(tp_tmp.visualize(min=0, max=1, palette=['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400','#FF0000']))

In [13]:
def click(b):
    point = ee.Geometry.Point(lon,lat)
    tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()
        
    header_tp = tp_point[0]
    data_tp = tp_point[1:]
    ydata_tp = [row[4]*1000 for row in data_tp]
    
    header_t2m = t2m_point[0]
    data_t2m = t2m_point[1:]
    ydata_t2m = [row[4]-273.2 for row in data_t2m]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat,2)) + ' / '+ str(round(lon,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm"
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[0,max(ydata_t2m)+2]
        )
    )

    fig = go.Figure(data=data, layout=layout)
    with out:
        clear_output(wait=True)
        display(py.iplot(fig,filename='test'))


In [33]:
point = ee.Geometry.Point(lon,lat)
tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()
data_tp = tp_point[1:]

In [48]:
df = pd.DataFrame(data_tp)
df

,0,1,2,3,4
0,0,8.549716,21.139604,None,0.000761
1,1,8.549716,21.139604,None,0.000328
2,2,8.549716,21.139604,None,0.000339
3,3,8.549716,21.139604,None,0.000185
4,4,8.549716,21.139604,None,0.000990
5,5,8.549716,21.139604,None,0.000625
6,6,8.549716,21.139604,None,0.000297
7,7,8.549716,21.139604,None,0.001432
8,8,8.549716,21.139604,None,0.000311
9,9,8.549716,21.139604,None,0.000186


In [35]:
alt.Chart(data_tp).mark_bar().encode(
    alt.X('month()'),
    y='4'
)

/Users/julia_wagemann/.virtualenvs/myvenv/lib/python3.7/site-packages/altair/vegalite/v4/api.py:96: UserWarning: data of type <class 'list'> not recognized
  warnings.warn("data of type {} not recognized".format(type(data)))


ValueError: precipitation encoding field is specified without a type; the type cannot be automatically inferred because the data is not specified as a pandas.DataFrame.

alt.Chart(...)

In [ ]:
def click(b):
    point = ee.Geometry.Point(lon,lat)
    tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()
        
    header_tp = tp_point[0]
    data_tp = tp_point[1:]
    ydata_tp = [row[4]*1000 for row in data_tp]
    
    header_t2m = t2m_point[0]
    data_t2m = t2m_point[1:]
    ydata_t2m = [row[4]-273.2 for row in data_t2m]
    


    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat,2)) + ' / '+ str(round(lon,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm"
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[0,max(ydata_t2m)+2]
        )
    )

    fig = go.Figure(data=data, layout=layout)
    with out:
        clear_output(wait=True)
        display(py.iplot(fig,filename='test'))


In [14]:
map1 = ipyleaflet.Map(
    zoom=2,
    layout={'height':'500px'},
)

map1.add_layer(ipyleaflet.TileLayer(url=img_t2m))
map1.add_layer(ipyleaflet.TileLayer(url=img_tp))
map1.add_control(ipyleaflet.LayersControl())

control = FullScreenControl()
map1.add_control(control)

map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [15]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        global lat, lon
        mark = ipyleaflet.Marker(location=kwargs.get('coordinates'))
        map1.add_layer(mark)
        location = mark.location
        lat, lon = location[0], location[1]        

map1.on_interaction(handle_click)

In [16]:
out=widgets.Output()
button=widgets.Button(description='Plot climate graph')
button.on_click(click)
display(out)
display(button)

Output()

Button(description='Plot climate graph', style=ButtonStyle())

NameError: name 'go' is not defined

<br>

<a href="./00_index.ipynb"><< Index - Dashboarding with Jupyter Notebooks and Voila </a><span style="float:right;"><a href="./02_voila_dashboards.ipynb"> 02 - Introduction to Voila dashboards >></a></span>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>